# Semantisk Kjerne

I dette kodeeksempelet vil du bruke [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI-rammeverket for å lage en enkel agent.

Målet med dette eksempelet er å vise deg trinnene som vi senere vil bruke i de ekstra kodeeksemplene når vi implementerer de ulike agentmønstrene.


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Opprette klienten

I dette eksempelet vil vi bruke [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for tilgang til LLM.

`ai_model_id` er definert som `gpt-4o-mini`. Prøv å endre modellen til en annen modell som er tilgjengelig på GitHub Models-markedsplassen for å se forskjellige resultater.

For at vi skal kunne bruke `Azure Inference SDK`, som brukes for `base_url` for GitHub Models, vil vi bruke `OpenAIChatCompletion`-koblingen innen Semantic Kernel. Det finnes også andre [tilgjengelige koblinger](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) for å bruke Semantic Kernel med andre modellleverandører.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Opprette agenten

Nedenfor oppretter vi agenten kalt `TravelAgent`.

I dette eksempelet bruker vi veldig enkle instruksjoner. Du kan endre disse instruksjonene for å se hvordan agenten reagerer annerledes.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Kjøre agenten

Nå kan vi kjøre agenten ved å definere en tråd av typen `ChatHistoryAgentThread`. Eventuelle nødvendige systemmeldinger gis til agentens invoke_stream-argument `messages`.

Når disse er definert, oppretter vi en `user_inputs` som representerer det brukeren sender til agenten. I dette tilfellet har vi satt denne meldingen til `Planlegg en solfylt ferie for meg`.

Du kan gjerne endre denne meldingen for å se hvordan agenten reagerer forskjellig.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
